In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().config("spark.network.timeout", '200s').getOrCreate()
from pyspark.sql.types import *

In [0]:
!wget "https://nycopendata.socrata.com/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD"

--2019-05-06 00:00:26--  https://nycopendata.socrata.com/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD
Resolving nycopendata.socrata.com (nycopendata.socrata.com)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to nycopendata.socrata.com (nycopendata.socrata.com)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [  <=>               ]  10.89G  5.01MB/s    in 40m 27s 

Last-modified header invalid -- time-stamp ignored.
2019-05-06 00:40:53 (4.60 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [11696132217]



In [0]:
!ls

 derby.log	 metastore_db			 sample_data
 fhrw-4uyv.csv	'rows.csv?accessType=DOWNLOAD'	 spark-2.3.3-bin-hadoop2.7


In [0]:
!cat rows.csv\?accessType\=DOWNLOAD | head

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
26578637,10/28/2013 12:00:00 AM,11/01/2013 12:00:00 AM,HPD,Department of Housing Preservation and Development,HEATING,HEAT,RESIDENTIAL BUILDING,10466,3908 BRONX BOULEVARD,BRONX BOULEVARD,EAST 222 STREET,EAST 224 STREET,,,ADDRESS,BRONX,,N/A,Closed,,"The Department of Housing Preservation and Development responded to a complaint of no heat or hot water and was advised by a tenan

In [0]:
!head -1 rows.csv?accessType=DOWNLOAD

Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location


In [0]:
columns = """Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate,Y Coordinate,Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location"""

In [0]:
columns = columns.replace(" ",'_').replace('\n','').lower().split(",")
columns[:10]  # just showing the first few column names

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address']

In [0]:
nyc_schema = StructType([StructField(colName, StringType(), True) for colName in columns])

In [0]:
print(nyc_schema)

StructType(List(StructField(unique_key,StringType,true),StructField(created_date,StringType,true),StructField(closed_date,StringType,true),StructField(agency,StringType,true),StructField(agency_name,StringType,true),StructField(complaint_type,StringType,true),StructField(descriptor,StringType,true),StructField(location_type,StringType,true),StructField(incident_zip,StringType,true),StructField(incident_address,StringType,true),StructField(street_name,StringType,true),StructField(cross_street_1,StringType,true),StructField(cross_street_2,StringType,true),StructField(intersection_street_1,StringType,true),StructField(intersection_street_2,StringType,true),StructField(address_type,StringType,true),StructField(city,StringType,true),StructField(landmark,StringType,true),StructField(facility_type,StringType,true),StructField(status,StringType,true),StructField(due_date,StringType,true),StructField(resolution_description,StringType,true),StructField(resolution_action_updated_date,StringType,t

In [0]:
nyc311 = spark.read.csv("rows.csv?accessType=DOWNLOAD",
                         header = True, 
                         schema = nyc_schema)

In [0]:
nyc311.select('created_date','agency_name','city').show(10, truncate = False)

+----------------------+--------------------------------------------------+------------+
|created_date          |agency_name                                       |city        |
+----------------------+--------------------------------------------------+------------+
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|BRONX       |
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|BROOKLYN    |
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|BRONX       |
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|BRONX       |
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|Far Rockaway|
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|Jamaica     |
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|BROOKLYN    |
|10/28/2013 12:00:00 AM|Department of Housing Preservation and Development|BROOKLYN    |
|10/28/2013 04:51:48 

In [0]:
nyc311.count()

20589056

In [0]:
nyc311.show(5)

+----------+--------------------+--------------------+------+--------------------+--------------+----------+--------------------+------------+--------------------+-----------------+---------------+-----------------+---------------------+---------------------+------------+------------+--------+-------------+------+--------+----------------------+------------------------------+------------------+----------+--------+------------+------------+----------------------+------------------+------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+-----------------+------------------+--------------------+
|unique_key|        created_date|         closed_date|agency|         agency_name|complaint_type|descriptor|       location_type|incident_zip|    incident_address|      street_name| cross_street_1|   cross_street_2|intersection_street_1|intersection_street_2|address_type|        city|landmark|facility_type|s

In [0]:
from datetime import datetime
from pyspark.sql.functions import col,udf, year

to_date_time =  udf(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'), TimestampType())
nyc311_2 = nyc311.withColumn('created_date', to_date_time(col('created_date'))).withColumn('year', year('created_date'))
nyc311_2.select(['created_date', 'year']).show(10)

+-------------------+----+
|       created_date|year|
+-------------------+----+
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 00:00:00|2013|
|2013-10-28 16:51:48|2013|
|2013-10-28 16:50:27|2013|
+-------------------+----+
only showing top 10 rows



In [0]:
nyc311_2.sample(False, 0.5, 42).createOrReplaceTempView('nyc311_table')

In [0]:
create_partitioned_table_sql = """CREATE TABLE if not exists nyc311_orc_partitioned (unique_key STRING, created_date STRING, closed_date STRING, agency STRING, agency_name STRING, complaint_type STRING, descriptor STRING, location_type STRING, incident_zip STRING, incident_address STRING, street_name STRING, cross_street_1 STRING, cross_street_2 STRING, intersection_street_1 STRING, intersection_street_2  STRING, address_type STRING, city  STRING, landmark STRING, facility_type STRING, status STRING, due_date STRING, resolution_description  STRING,resolution_action_updated_date STRING, community_board STRING, bbl STRING, borough STRING, x_coordinate STRING, y_coordinate STRING, open_data_channel_type STRING, park_facility_name STRING, park_borough STRING, vehicle_type STRING, taxi_company_borough STRING, taxi_pick_up_location STRING, bridge_highway_name STRING, bridge_highway_direction  STRING, road_ramp STRING, bridge_highway_segment STRING, latitude STRING, longitude STRING, location STRING) PARTITIONED BY (year INT) STORED AS ORC """

In [0]:
spark.sql(create_partitioned_table_sql)

DataFrame[]

In [0]:
set_dynamic_mode  = "SET hive.exec.dynamic.partition.mode = nonstrict"
spark.sql(set_dynamic_mode)

DataFrame[key: string, value: string]

In [0]:
insert_sql = """INSERT INTO nyc311_orc_partitioned PARTITION (year) SELECT unique_key, created_date, closed_date, agency, agency_name, complaint_type, descriptor, location_type, incident_zip, incident_address, street_name, cross_street_1, cross_street_2, intersection_street_1, intersection_street_2, address_type, city, landmark, facility_type, status, due_date, resolution_description, resolution_action_updated_date, community_board, bbl, borough, x_coordinate, y_coordinate, open_data_channel_type, park_facility_name, park_borough, vehicle_type, taxi_company_borough, taxi_pick_up_location, bridge_highway_name, bridge_highway_direction, road_ramp, bridge_highway_segment, latitude, longitude, location, year FROM nyc311_table"""

In [0]:
spark.sql(insert_sql)

DataFrame[]

In [0]:
spark.sql("show partitions nyc311_orc_partitioned").show()

+---------+
|partition|
+---------+
|year=2010|
|year=2011|
|year=2012|
|year=2013|
|year=2014|
|year=2015|
|year=2016|
|year=2017|
|year=2018|
|year=2019|
+---------+



In [0]:
spark.sql('create table nyc311_orc_unpartitioned stored as orc as select * from nyc311_table')

DataFrame[]

In [0]:
import os

In [0]:
os.listdir("spark-warehouse/nyc311_orc_unpartitioned")[:10]

['part-00016-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000',
 '.part-00059-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000.crc',
 'part-00054-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000',
 'part-00017-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000',
 '.part-00072-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000.crc',
 '.part-00038-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000.crc',
 'part-00071-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000',
 'part-00056-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000',
 '.part-00045-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000.crc',
 '.part-00017-fd414c7a-d2aa-4f51-b95d-69df19ad6dea-c000.crc']

In [0]:
os.listdir("spark-warehouse/nyc311_orc_partitioned")

['year=2015',
 'year=2019',
 'year=2018',
 'year=2016',
 'year=2011',
 'year=2010',
 'year=2013',
 'year=2014',
 'year=2012',
 'year=2017']

In [0]:
os.listdir("spark-warehouse/nyc311_orc_partitioned/year=2010")[:10]

['part-00074-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000',
 'part-00006-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000',
 '.part-00059-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000.crc',
 '.part-00026-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000.crc',
 'part-00018-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000',
 '.part-00022-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000.crc',
 'part-00061-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000',
 '.part-00071-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000.crc',
 '.part-00079-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000.crc',
 'part-00075-0acd40fa-5f20-4d69-b546-52e25aae62fb.c000']

In [0]:
import time
t0 = time.time()
aggregates = spark.sql("select agency_name, city, incident_zip, count(*) as frequency from nyc311_orc_unpartitioned where year = '2010' group by agency_name, city, incident_zip").toPandas()
t1 = time.time()

print('time taken with unpartitioned table ', round(t1-t0,2))

time taken with unpartitioned table  13.04


In [0]:
t0 = time.time()
aggregates = spark.sql("select agency_name, city,incident_zip, count(*) as frequency from nyc311_orc_partitioned where year = '2010' group by agency_name, city, incident_zip").toPandas()
t1 = time.time()

print('time taken with partitioned table ', round(t1-t0,2))

time taken with partitioned table  5.49
